In [15]:
import pydal
from pydal import DAL, Field
import random

In [16]:
db = DAL('sqlite://storage.sqlite')

In [17]:
db.define_table('student',
                Field('name', 'string'),
                Field('email', 'string'),
                Field('n_quarters', 'integer'),
                Field('gpa', 'double'),
)

db.commit()

In [18]:
def random_string(n):
    return ''.join(random.choice('abcdefghijklmnopqrstuvwxyz') for _ in range(n))

In [19]:
for _ in range(10):
    db.student.insert(
        name=random_string(10),
        email=random_string(10) + '@example.com',
        n_quarters=random.randint(1, 10),
        gpa=random.uniform(0, 4),
    )
db.commit()

In [20]:
rows = db(db.student.gpa > 2).select(db.student.email, db.student.gpa, orderby=~db.student.gpa)

In [21]:
emails = [(row.email, row.gpa) for row in rows]

In [22]:
emails

[('ukffhsbgtr@example.com', 3.677175088573455),
 ('jmkeutzirh@example.com', 3.342002192350522),
 ('boygdabccm@example.com', 3.182831060699003),
 ('qckydgjzbe@example.com', 3.088173586350787),
 ('iztmsaebur@example.com', 3.046844885850957),
 ('ewetkkyptk@example.com', 3.0391938358986907),
 ('melnlyucxk@example.com', 2.4976518708025495)]

In [27]:
# Update method 1.
student1 = 'oqkpvvkeqc@example.com'
student2 = 'nyrzzjdmlg@example.com'

s1 = db(db.student).select().first()
s1.gpa = 4
s1.update_record()

<Row {'id': 1, 'name': 'jjppsaernw', 'email': 'dqadxkoktf@example.com', 'n_quarters': 1, 'gpa': 4}>

In [28]:
db(db.student.email == student2).update(gpa=4)

0

In [29]:
db(db.student.gpa < 2).delete()
db.commit()

## Representing relations

In [30]:
db.define_table('sighting',
    Field('date_of_sighting', 'date'),
    Field('location', 'string'),
    Field('species', 'string'),
    Field('count', 'integer'),
)
db.commit()

In [31]:
db.sighting.insert(
    date_of_sighting='2021-05-01',
    location='Seattle',
    species='Robin',
    count=1,
)
db.sighting.insert(
    date_of_sighting='2021-05-01',
    location='Seattle',
    species='Turkey',
    count=10,
)
db.commit()

## Birdwatching tables

In [32]:
db_birds = DAL('sqlite://birds.sqlite')

In [33]:
db_birds.define_table(
    'outing', # This table has an id as its key. 
    Field('date_of_outing', 'date'),
    Field('location', 'string'),
    Field('weather', 'string'),
    )

db_birds.define_table(
    'species',
    Field('english_name', 'string'),
    Field('latin_name', 'string'),
    Field('is_threatened', 'boolean'),
)

# This is a many to many relation. 
# A species is seen in many outings. 
# Any (outing, species) pair can have many sightings, each with a count. 
db_birds.define_table(
    'sighting',
    Field('outing_id', 'reference outing'), # Called a "foreign key"
    Field('species_id', 'reference species'), # Called a "foreign key"
    Field('count', 'integer'),
    )

db_birds.commit()

## People db

In [34]:
db_people = DAL('sqlite://people.sqlite')

db_people.define_table(
    'person', 
    Field('name', 'string'),
    Field('email', 'string'),
    Field('mob', 'integer'),
    Field('dobim', 'integer'),
    Field('partner', 'reference person', ondelete='SET NULL'),
    )

db_people.define_table(
    'invitation',
    Field('from_person', 'reference person'),
    Field('to_person',), # email 
    Field('event_name'),
)

db_people.commit()

### Inserting a checklist

In [35]:
import datetime

In [36]:
date = datetime.date.today()
location = 'Santa Cruz'
weather = 'Sunny'
what = [
    ('Robin', 3),
    ('Turkey', 2),
    ('Sparrow', 1)
]

In [37]:
# First, insert the outing.
outing_id = db_birds.outing.insert(
    date_of_outing=date,
    location=location,
    weather=weather,
)

In [38]:
# Insert the sightings. 
for species_name, count in what:
    # Gets the species id, inserting the species if needed.
    species = db_birds(db_birds.species.english_name == species_name).select().first()
    if species is None:
        species_id = db_birds.species.insert(
            english_name=species_name,
            latin_name='Unknown',
            is_threatened=False,
        )
    else:
        species_id = species.id
    # Insert the sighting.
    db_birds.sighting.insert(
        outing_id=outing_id,
        species_id=species_id,
        count=count,
    )
db_birds.commit()

## Reading information across multiple tables

In [39]:
rows = db_birds(
    (db_birds.sighting.outing_id == db_birds.outing.id) & # join 1
    (db_birds.sighting.species_id == db_birds.species.id) # join 2
).select().as_list()


In [40]:
rows

[{'sighting': {'id': 1, 'outing_id': 1, 'species_id': 1, 'count': 3},
  'species': {'id': 1,
   'english_name': 'Robin',
   'latin_name': 'Unknown',
   'is_threatened': False},
  'outing': {'id': 1,
   'date_of_outing': datetime.date(2024, 10, 28),
   'location': 'Santa Cruz',
   'weather': 'Sunny'}},
 {'sighting': {'id': 2, 'outing_id': 1, 'species_id': 2, 'count': 2},
  'species': {'id': 2,
   'english_name': 'Turkey',
   'latin_name': 'Unknown',
   'is_threatened': False},
  'outing': {'id': 1,
   'date_of_outing': datetime.date(2024, 10, 28),
   'location': 'Santa Cruz',
   'weather': 'Sunny'}},
 {'sighting': {'id': 3, 'outing_id': 1, 'species_id': 3, 'count': 1},
  'species': {'id': 3,
   'english_name': 'Sparrow',
   'latin_name': 'Unknown',
   'is_threatened': False},
  'outing': {'id': 1,
   'date_of_outing': datetime.date(2024, 10, 28),
   'location': 'Santa Cruz',
   'weather': 'Sunny'}},
 {'sighting': {'id': 4, 'outing_id': 2, 'species_id': 1, 'count': 3},
  'species': {'id'

In [44]:
rows = db_birds(db_birds.species.id == 1).select()
for r in rows:
    print(r.english_name)

Robin


In [46]:
rows = db_birds(
    (db_birds.sighting.outing_id == db_birds.outing.id) & # join 1
    (db_birds.sighting.species_id == db_birds.species.id) # join 2
).select()



In [48]:
for r in rows:
    print(f"I saw {r.sighting.count} {r.species.english_name} on {r.outing.date_of_outing} in {r.outing.location}.")

I saw 3 Robin on 2024-10-28 in Santa Cruz.
I saw 2 Turkey on 2024-10-28 in Santa Cruz.
I saw 1 Sparrow on 2024-10-28 in Santa Cruz.
I saw 3 Robin on 2024-11-04 in Santa Cruz.
I saw 2 Turkey on 2024-11-04 in Santa Cruz.
I saw 1 Sparrow on 2024-11-04 in Santa Cruz.


## Registrar

In [ ]:
dbr = DAL('sqlite://ucsc.sqlite')

In [ ]:
dbr.define_table(
    'cat_course',
    Field('number', 'string'),
    Field('description', 'text'),
    Field('course_name', 'string'),   
)

dbr.define_table(
    'quarter',
    Field('season', 'string'),
    Field('year', 'integer'),
)

dbr.define_table(
    'instructor',
    Field('name', 'string'),
    Field('email', 'string'),
)

dbr.define_table(
    'offering',
    Field('course_id', 'reference cat_course'),
    Field('quarter_id', 'reference quarter'),
    Field('instructor_id', 'reference instructor'),
    Field('enrollment_limit', 'integer'),
)

dbr.define_table(
    'student',
    Field('name', 'string'),
    Field('email', 'string'),
)

dbr.define_table(
    'enrollment',
    Field('student_id', 'reference student'),
    Field('offering_id', 'reference offering'),
    Field('for_letter_grade', 'boolean'),
)

dbr.commit()